In [1]:
# https://towardsdatascience.com/how-to-gain-state-of-the-art-result-on-tabular-data-with-deep-learning-and-embedding-layers-d1eb6b83c52c

In [2]:
%load_ext autoreload
%autoreload 2

In [52]:
import pandas as pd
import numpy as np

In [4]:
from exp.nb_01 import load_data
from exp.nb_02 import gini_score

In [5]:
def write_down_features(df):
    train_df = df[0:n_train]
    test_df = df[n_train:]
    train_df['label'] = label_df
    train_df.to_csv('data/train_feature.csv', index=False)
    test_df.to_csv('data/test_feature.csv', index=False)

In [6]:
train_df, test_df = load_data(keep_df=True)
n_train, n_test = len(train_df), len(test_df)
n_train, n_test

/home/vule/miniconda3/envs/thanos/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3214: DtypeWarning: Columns (51) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):
/home/vule/miniconda3/envs/thanos/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3214: DtypeWarning: Columns (50) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


(30000, 20000)

In [7]:
label_df = train_df['label']
del train_df['label']

In [8]:
df = pd.concat([train_df, test_df])

In [9]:
# list(df.columns)

In [10]:
df[['FIELD_1', 'FIELD_2', 'FIELD_3', 'FIELD_4', 'FIELD_5', 'FIELD_6', 'FIELD_7', 'FIELD_8', 'FIELD_9', 'FIELD_10']]

,FIELD_1,FIELD_2,FIELD_3,FIELD_4,FIELD_5,FIELD_6,FIELD_7,FIELD_8,FIELD_9,FIELD_10
0,1,1.0,2547.0,0.0,0.0,0.0,[],NaN,na,NaN
1,1,0.0,-1.0,1.0,1.0,1.0,"['GD', 'GD', 'TE']",MALE,GD,T1
2,0,1.0,3273.0,1.0,4.0,0.0,"['DN', 'HN', 'DN']",FEMALE,DN,T1
3,0,1.0,3991.0,1.0,1.0,0.0,"['CH', 'TQ']",NaN,na,NaN
4,0,1.0,1450.0,0.0,2.0,0.0,"['SV', 'GD']",MALE,GD,T1
...,...,...,...,...,...,...,...,...,...,...
19995,1,1.0,1443.0,0.0,3.0,0.0,"['DN', 'DN']",FEMALE,DN,GH
19996,1,1.0,2898.0,0.0,0.0,0.0,[],NaN,na,NaN
19997,1,1.0,1450.0,0.0,0.0,0.0,[],NaN,na,NaN
19998,1,1.0,2176.0,2.0,2.0,0.0,"['TE', 'DN', 'DK', 'TE']",FEMALE,DN,T1


### Create isna columns

In [11]:
### create isna columns
columns = list(df.columns)
columns.remove('id')
for c in columns:
    df[f"{c}_isna"] = df[c].isna()
    
df['n_na'] = df.isnull().sum(axis=1)

### Lower and unicode all columns


In [12]:
columns = train_df.select_dtypes(exclude=['int', 'float', 'bool']).columns
for c in columns:
    not_na_id = df[c].notna()
    try:
        df.loc[not_na_id, c] = df[not_na_id][c].str.lower().apply(lambda x: re.sub(' +', ' ', unidecode.unidecode(x)))
    except BaseException as e:
        pass

### Province, District

In [13]:
import unidecode
import re

df['is_city'] = df.province.str.contains('thanh pho') | df.district.str.contains('thanh pho')

### Age_source 1 2

In [14]:
df['age_isdiff'] = df['age_source1'] != df['age_source2']
df['age1_less_18'] = df['age_source1'] < 18
df['age2_less_18'] = df['age_source2'] < 18

df['age1_gt_65'] = df['age_source1'] > 65
df['age2_gt_65'] = df['age_source2'] > 65

### maCv

In [15]:
from collections import Counter
from exp.nb_01 import cv_mapper

not_na_id = df.maCv.notna()
len(Counter(df['maCv']))
for k, v in cv_mapper.items():
    df.loc[not_na_id, 'maCv'] = df[not_na_id].maCv.apply(lambda x: v if k in x else x)
    
f_dict = dict(Counter(df['maCv']))
df.loc[not_na_id, 'maCv'] = df[not_na_id].maCv.apply(lambda x: 'rare_job' if f_dict[x] < 3 else x)

### FIELD

### FIELD_7

In [64]:
df['FIELD_7'] = df[['FIELD_7']].fillna('[]')

In [66]:
list(df['FIELD_7'][3])

["['CH', 'TQ']", '[]']

In [54]:
df['FIELD_7'][0:3].apply(lambda x: 0 if np.nan == x else len(x))

0     2
1    18
2    18
Name: FIELD_7, dtype: int64

In [36]:
[[] for _ in range(sum(~not_na_id))]

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],


In [18]:
df['len_f7'] = df['FIELD_7'].apply(lambda x: len(x))

TypeError: object of type 'float' has no len()

In [16]:
write_down_features(df)

/home/vule/miniconda3/envs/thanos/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
